# Default equation
def Eqn_Enz_MichaelisMenten_Activation_Allosteric(Conc_Enzyme, Conc_Substrate, Conc_Activator, kcat, KM, Ka, n):
    return (kcat * Conc_Enzyme * (1 + (Conc_Activator / Ka) ** n)) * (Conc_Substrate / (KM + Conc_Substrate))


In [26]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import Avogadro
from scipy import sqrt
from scipy.optimize import fixed_point
import sympy as sm

from fun import Plotly_Dynamics, generateRGBList

#plotting
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from random import randint
import os
import re
from plotly.subplots import make_subplots

# Micromolar conversion 
def uMtoCount(inVal:float):
    return inVal*10**-6*Avogadro
def counttouM(inVal:float):
    return inVal / Avogadro / 10**-6

In [2]:
# constants
SIM_DURATION_SECONDS = 10
STEPS_PER_SECOND = 100

TOTAL_STEPS = SIM_DURATION_SECONDS * STEPS_PER_SECOND

# kcats
KCAT_HEXOKINASE = 4e3
KCAT_PFK = 1.7e4
KCAT_PFK_ACTIVE = 1.7e4
KCAT_PFK_INACTIVE = 1.7e4
KCAT_PK = 170
##
KCAT_SUMMARY_MAKE_PEPASE = 1e4 # Arbitrary
# ks
K_SUMMARY_MAKE_PEPASE = 1e4

# KMs
KM_HEXOKINASE_GLUCOSE = uMtoCount(5)
KM_PFK_ACTIVE = uMtoCount(0.21)
KM_PFK_INACTIVE = uMtoCount(3.1e7)
KM_PK_PEP = uMtoCount(700)

#####
KM_SUMMARY_MAKE_PEPASE = uMtoCount(30) # just guessing.

# Concentrations
## Metabolites
CONC_GLUCOSE_T0 = uMtoCount(500)
CONC_ADP_T0 = uMtoCount(1700)
CONC_ATP_T0 = uMtoCount(5000)
## Intermediates
CONC_ATP = CONC_ATP_T0
CONC_ADP = CONC_ADP_T0
CONC_F6P = uMtoCount(15)
CONC_F16P = 0
CONC_PEP = 0
CONC_PYRUVATE = 0
## Enzymes
CONC_PFK = 4000 / Avogadro
CONC_PFK_ACTIVE = 0
CONC_PFK_INACTIVE = 0
CONC_HEXOKINASE = uMtoCount(0.01)
CONC_SUMMARY_MAKE_PEPASE = uMtoCount(1)
CONC_PYRUVATE_KINASE = uMtoCount(1)
CONC_PHOSPHOGLUCOSEISOMERASE = uMtoCount(1)

## Current exploratory values
KI_ATP_PYRUVATEKINASE = uMtoCount(5000)

In [31]:
def saturation(conc, KM):
    return (conc / (KM + conc))

In [32]:
saturation(CONC_GLUCOSE_T0, KM=KM_HEXOKINASE_GLUCOSE)

0.9900990099009901

In [30]:
CONC_GLUCOSE_T0 / (CONC_GLUCOSE_T0 + KM_HEXOKINASE_GLUCOSE)

0.9900990099009901

In [3]:
useOneStatePFK = True

def saturation(conc, KM):
    return (conc / (KM + conc))

# Attempting to base
def hexokinase(Conc_Substrate = CONC_GLUCOSE_T0, Conc_Enzyme = CONC_HEXOKINASE, kcat = KCAT_HEXOKINASE, KM = KM_HEXOKINASE_GLUCOSE):
    # Note: glucose is not decreasing...
    return (kcat * Conc_Enzyme * saturation(Conc_Substrate, KM))

if useOneStatePFK:
    def pfk(Conc_Substrate = CONC_F6P, Conc_Enzyme = CONC_PFK, kcat = KCAT_PFK_ACTIVE, KM = KM_PFK_ACTIVE):
        return (kcat * Conc_Enzyme * saturation(Conc_Substrate, KM))
else:
    def pfk_active(Conc_Substrate = CONC_F6P, Conc_Enzyme = CONC_PFK_ACTIVE, kcat = KCAT_PFK_ACTIVE, KM = KM_PFK_ACTIVE):
        return (kcat * Conc_Enzyme *  saturation(Conc_Substrate, KM))
    def pfk_inactive(Conc_Substrate = CONC_F6P, Conc_Enzyme = CONC_PFK_INACTIVE, kcat = KCAT_PFK_INACTIVE, KM = KM_PFK_INACTIVE):
        return (kcat * Conc_Enzyme * saturation(Conc_Substrate, KM))

def summaryMakePEPase(Conc_Substrate = CONC_F16P,Conc_Enzyme = CONC_SUMMARY_MAKE_PEPASE, kcat = KCAT_SUMMARY_MAKE_PEPASE, KM = KM_SUMMARY_MAKE_PEPASE):
    return(kcat * Conc_Enzyme * saturation(Conc_Substrate, KM))

def pyruvateKinase(Conc_Substrate = CONC_PEP, Conc_Inhibitor = CONC_ATP, Ki = KI_ATP_PYRUVATEKINASE, Conc_Enzyme = CONC_PYRUVATE_KINASE, kcat = KCAT_PK, KM = KM_PK_PEP):
    return (kcat * Conc_Enzyme / (1 + (Conc_Inhibitor / Ki)) * saturation(Conc_Substrate, KM))

Need to add:
- % Substrate Saturation
- % maximum activity
- step-turnover

-- percent of steady state concentration?

In [15]:
# reimplement simulation: 
metaboliteLegend = ['F6P', 'F16BP', 'PEP', 'pyruvate']
enzymeLegend = ['HK', 'PFK', 'PS', 'PK']

# For now only worry about F6P, F16BP, PEP, pyruvate
arr2DCounts= np.zeros([TOTAL_STEPS, 4]) 
arr2DEnzymePercentSubstrateSaturation = np.zeros([TOTAL_STEPS, 4])
arr2DEnzymePercentMaxActivty = np.zeros([TOTAL_STEPS, 4])
# 4 enzymes atm
arr2DEnzymeStepTurnover = np.zeros([TOTAL_STEPS, 4])

for step in range(1,TOTAL_STEPS):
    # Get turnover for each enzyme
    arr2DEnzymeStepTurnover[step][0] = hexokinase() / STEPS_PER_SECOND                               # In: Gluc  ; Out: F6P
    arr2DEnzymeStepTurnover[step][1] = pfk(arr2DCounts[step-1][0]) / STEPS_PER_SECOND                 # In: F6P   ; Out: F16BP
    arr2DEnzymeStepTurnover[step][2] = summaryMakePEPase(arr2DCounts[step-1][1]) / STEPS_PER_SECOND   # In: F16BP ; Out: PEP
    arr2DEnzymeStepTurnover[step][3] = pyruvateKinase(arr2DCounts[step-1][2]) / STEPS_PER_SECOND      # In: PEP   ; Out: Pyruvate

    # Get percent substrate Saturation:
    arr2DEnzymePercentSubstrateSaturation[step] = saturation(CONC_GLUCOSE_T0, KM_HEXOKINASE_GLUCOSE)
    arr2DEnzymePercentSubstrateSaturation[step] = saturation(arr2DCounts[step-1][0], KM_PFK_ACTIVE)
    arr2DEnzymePercentSubstrateSaturation[step] = saturation(arr2DCounts[step-1][1], KM_SUMMARY_MAKE_PEPASE)
    arr2DEnzymePercentSubstrateSaturation[step] = saturation(arr2DCounts[step-1][2], KM_PK_PEP)

    # Get Percent Max Activity
    arr2DEnzymePercentMaxActivty[step][0] = round(arr2DEnzymeStepTurnover[step][0] / (KCAT_HEXOKINASE * CONC_HEXOKINASE) * 100 * STEPS_PER_SECOND, 2)
    arr2DEnzymePercentMaxActivty[step][1] = round(arr2DEnzymeStepTurnover[step][1] / (KCAT_PFK * CONC_PFK) * 100 * STEPS_PER_SECOND, 2)
    arr2DEnzymePercentMaxActivty[step][2] = round(arr2DEnzymeStepTurnover[step][2] / (KCAT_SUMMARY_MAKE_PEPASE * CONC_SUMMARY_MAKE_PEPASE) * 100 * STEPS_PER_SECOND, 2)
    arr2DEnzymePercentMaxActivty[step][3] = round(arr2DEnzymeStepTurnover[step][3] / (KCAT_PK * CONC_PYRUVATE_KINASE) * 100 * STEPS_PER_SECOND, 2)

    # Update counts
    arr2DCounts[step][0] = max(0, arr2DCounts[step-1][0] + arr2DEnzymeStepTurnover[step][0] - arr2DEnzymeStepTurnover[step][1])
    arr2DCounts[step][1] = max(0, arr2DCounts[step-1][1] + arr2DEnzymeStepTurnover[step][1] - arr2DEnzymeStepTurnover[step][2])
    arr2DCounts[step][2] = max(0, arr2DCounts[step-1][2] + arr2DEnzymeStepTurnover[step][2] - arr2DEnzymeStepTurnover[step][3])
    arr2DCounts[step][3] = max(0, arr2DCounts[step-1][3] + arr2DEnzymeStepTurnover[step][3])



In [20]:
def Plotly_EnzymeTurnover(Title, Time, Data, Legend):
    ''' Generate a plotly plot wiht time on x axis and colored line for each data'''
    X = Time
    Y = Data
    # Temp conversion to uM
    Y = np.multiply(Y, 1e6)
    Y = np.divide(Y, Avogadro)

    colors = generateRGBList(len(Legend))

    #assert len(X) == Y.shape[-1]

    # make figure
    fig = go.Figure()

    for i in range(len(Legend)):
        if Legend[i] == "Pseudo":
            continue
        fig.add_trace(
            # The line
            go.Scatter(
                x = X, y = Y[i],
                mode = 'lines', name = Legend[i],
                line = dict(color = colors[i]),
                connectgaps = True,
                ))
        # Points at t0 and tn (start and end)
        fig.add_trace(go.Scatter(
            x=[X[0], X[-1]],
            y=[Y[i][0], Y[i][-1]],
            mode='markers', name = Legend[i],
            marker = dict(color = colors[i]),
            showlegend = False
        ))

    # Formatting 
    fig.update_layout(
        # xaxis
        xaxis=dict(
            title = "Time (s)",
            titlefont = dict(
                family = 'Arial',
                size = 16,
                color = 'rgb(0,0,0)'
            ),
            showline=True,
            showgrid=True,
            showticklabels=True,
            linecolor='rgb(0, 0, 0)',
            linewidth=2, 
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=16,
                color='rgb(0, 0, 0)',
            ),
        ),
        #  yaxis
        yaxis=dict(
            title = "Enzyme Turnover / Step (uM)",
            titlefont = dict(
                family = 'Arial',
                size = 16,
                color = 'rgb(0,0,0)'
            ),
            zeroline = True,
            showgrid=True,
            showline=True,
            showticklabels=True,
            linecolor='rgb(0, 0, 0)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=16,
                color='rgb(0, 0, 0)',
            )
        ),
        autosize=True,   
        legend = dict(
            orientation = "h",
        ),
        plot_bgcolor='white'
        )

    annotations = []
    # Adding labels
    for y_trace, label in zip(Y, Legend):
        # labeling the left_side of the plot
        annotations.append(dict(xref='paper', axref = 'x', x=0.1, y=y_trace[0]+.1,
                                    xanchor='right', yanchor='middle',
                                    text=' {:2g}mM'.format(round(y_trace[0], 2)),
                                    font=dict(family='Arial',
                                                size=16),
                                    showarrow=False))
        # labeling the right_side of the plot
        annotations.append(dict(xref='paper', axref = "x", x=0.95, y=y_trace[-1],
                                    xanchor='left', yanchor='middle',
                                    text=label +' {:2g}mM'.format(round(y_trace[-1], 2)),
                                    font=dict(family='Arial',
                                                size=16),
                                    showarrow=False))
    # Title
    annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                                xanchor='left', yanchor='bottom',
                                text='[Metabolites] over time',
                                font=dict(family='Arial',
                                            size=30,
                                            color='rgb(37,37,37)'),
                                showarrow=False))

    fig.update_layout(annotations=annotations)
    return fig


In [21]:
def Plotly_EnzymeSubstrateSaturation(Title, Time, Data, Legend):
    ''' Generate a plotly plot wiht time on x axis and colored line for each data'''
    X = Time
    Y = Data

    colors = generateRGBList(len(Legend))

    #assert len(X) == Y.shape[-1]

    # make figure
    fig = go.Figure()

    for i in range(len(Legend)):
        if Legend[i] == "Pseudo":
            continue
        fig.add_trace(
            # The line
            go.Scatter(
                x = X, y = Y[i],
                mode = 'lines', name = Legend[i],
                line = dict(color = colors[i]),
                connectgaps = True,
                ))
        # Points at t0 and tn (start and end)
        fig.add_trace(go.Scatter(
            x=[X[0], X[-1]],
            y=[Y[i][0], Y[i][-1]],
            mode='markers', name = Legend[i],
            marker = dict(color = colors[i]),
            showlegend = False
        ))

    # Formatting 
    fig.update_layout(
        # xaxis
        xaxis=dict(
            title = "Time (s)",
            titlefont = dict(
                family = 'Arial',
                size = 16,
                color = 'rgb(0,0,0)'
            ),
            showline=True,
            showgrid=True,
            showticklabels=True,
            linecolor='rgb(0, 0, 0)',
            linewidth=2, 
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=16,
                color='rgb(0, 0, 0)',
            ),
        ),
        #  yaxis
        yaxis=dict(
            title = "Enzyme Substrate Saturation (%)",
            titlefont = dict(
                family = 'Arial',
                size = 16,
                color = 'rgb(0,0,0)'
            ),
            zeroline = True,
            showgrid=True,
            showline=True,
            showticklabels=True,
            linecolor='rgb(0, 0, 0)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=16,
                color='rgb(0, 0, 0)',
            )
        ),
        autosize=True,   
        legend = dict(
            orientation = "h",
        ),
        plot_bgcolor='white'
        )

    annotations = []
    # Adding labels
    for y_trace, label in zip(Y, Legend):
        # labeling the left_side of the plot
        annotations.append(dict(xref='paper', axref = 'x', x=0.1, y=y_trace[0]+.1,
                                    xanchor='right', yanchor='middle',
                                    text=' {:2g}mM'.format(round(y_trace[0], 2)),
                                    font=dict(family='Arial',
                                                size=16),
                                    showarrow=False))
        # labeling the right_side of the plot
        annotations.append(dict(xref='paper', axref = "x", x=0.95, y=y_trace[-1],
                                    xanchor='left', yanchor='middle',
                                    text=label +' {:2g}mM'.format(round(y_trace[-1], 2)),
                                    font=dict(family='Arial',
                                                size=16),
                                    showarrow=False))
    # Title
    annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                                xanchor='left', yanchor='bottom',
                                text='[Metabolites] over time',
                                font=dict(family='Arial',
                                            size=30,
                                            color='rgb(37,37,37)'),
                                showarrow=False))

    fig.update_layout(annotations=annotations)
    return fig


In [22]:
def Plotly_EnzymePercentMaxActivity(Title, Time, Data, Legend):
    ''' Generate a plotly plot wiht time on x axis and colored line for each data'''
    X = Time
    Y = Data

    colors = generateRGBList(len(Legend))

    #assert len(X) == Y.shape[-1]

    # make figure
    fig = go.Figure()

    for i in range(len(Legend)):
        if Legend[i] == "Pseudo":
            continue
        fig.add_trace(
            # The line
            go.Scatter(
                x = X, y = Y[i],
                mode = 'lines', name = Legend[i],
                line = dict(color = colors[i]),
                connectgaps = True,
                ))
        # Points at t0 and tn (start and end)
        fig.add_trace(go.Scatter(
            x=[X[0], X[-1]],
            y=[Y[i][0], Y[i][-1]],
            mode='markers', name = Legend[i],
            marker = dict(color = colors[i]),
            showlegend = False
        ))

    # Formatting 
    fig.update_layout(
        # xaxis
        xaxis=dict(
            title = "Time (s)",
            titlefont = dict(
                family = 'Arial',
                size = 16,
                color = 'rgb(0,0,0)'
            ),
            showline=True,
            showgrid=True,
            showticklabels=True,
            linecolor='rgb(0, 0, 0)',
            linewidth=2, 
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=16,
                color='rgb(0, 0, 0)',
            ),
        ),
        #  yaxis
        yaxis=dict(
            title = "Enzyme Activity (%)",
            titlefont = dict(
                family = 'Arial',
                size = 16,
                color = 'rgb(0,0,0)'
            ),
            zeroline = True,
            showgrid=True,
            showline=True,
            showticklabels=True,
            linecolor='rgb(0, 0, 0)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=16,
                color='rgb(0, 0, 0)',
            )
        ),
        autosize=True,   
        legend = dict(
            orientation = "h",
        ),
        plot_bgcolor='white'
        )

    annotations = []
    # Adding labels
    for y_trace, label in zip(Y, Legend):
        # labeling the left_side of the plot
        annotations.append(dict(xref='paper', axref = 'x', x=0.1, y=y_trace[0]+.1,
                                    xanchor='right', yanchor='middle',
                                    text=' {:2g}mM'.format(round(y_trace[0], 2)),
                                    font=dict(family='Arial',
                                                size=16),
                                    showarrow=False))
        # labeling the right_side of the plot
        annotations.append(dict(xref='paper', axref = "x", x=0.95, y=y_trace[-1],
                                    xanchor='left', yanchor='middle',
                                    text=label +' {:2g}mM'.format(round(y_trace[-1], 2)),
                                    font=dict(family='Arial',
                                                size=16),
                                    showarrow=False))
    # Title
    annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                                xanchor='left', yanchor='bottom',
                                text='[Metabolites] over time',
                                font=dict(family='Arial',
                                            size=30,
                                            color='rgb(37,37,37)'),
                                showarrow=False))

    fig.update_layout(annotations=annotations)
    return fig


In [18]:
Plotly_Dynamics(Title = 'Counts', Time = np.linspace(0, SIM_DURATION_SECONDS, TOTAL_STEPS), Data = arr2DCounts.T, Legend = metaboliteLegend)

In [23]:
Plotly_EnzymeTurnover(Title = 'Enzyme Turnover', Time = np.linspace(0, SIM_DURATION_SECONDS, TOTAL_STEPS), Data = arr2DEnzymeStepTurnover.T, Legend = enzymeLegend)

In [24]:
Plotly_EnzymeSubstrateSaturation(Title = 'Enzyme Substrate Saturation', Time = np.linspace(0, SIM_DURATION_SECONDS, TOTAL_STEPS), Data = arr2DEnzymePercentSubstrateSaturation.T, Legend = enzymeLegend)

In [25]:
Plotly_EnzymePercentMaxActivity(Title = 'Enzyme Percent Max Activity', Time = np.linspace(0, SIM_DURATION_SECONDS, TOTAL_STEPS), Data = arr2DEnzymePercentMaxActivty.T, Legend = enzymeLegend)

In [29]:
def Plotly_Simulation_Overview(Time, metaboliteCountData, enzymeTurnoverData, enzymeSubstrateSaturationData, enzymePercentMaxActivityData, enzymeLegend, metaboliteLegend):
    X = Time
    # convert to uM
    metaboliteCountData = np.multiply(metaboliteCountData, 1e6) 
    metaboliteCountData = np.divide(metaboliteCountData, Avogadro)
    # uM / step
    enzymeTurnoverData = np.multiply(enzymeTurnoverData, 1e6)
    enzymeTurnoverData = np.divide(enzymeTurnoverData, Avogadro)

    Y = [metaboliteCountData, enzymeTurnoverData, enzymeSubstrateSaturationData, enzymePercentMaxActivityData]


    metaboliteColors = generateRGBList(len(metaboliteLegend))
    enzymeColors = generateRGBList(len(enzymeLegend))

    fig = make_subplots(
        rows = 2, cols = 2, shared_xaxes= True,
        subplot_titles=("[metabolites]", "Enzyme: Turnover (uM/step)", "Enzyme: Percent Substrate Saturation", "Enzyme: Percent Max Activity")
    )


    # [metabolites]
    for i in range(len(metaboliteLegend)):
        fig.add_trace(
            # The line
            go.Scatter(
                x = X, y = Y[0][i],
                mode = 'lines', name = metaboliteLegend[i],
                line = dict(color = metaboliteColors[i]),
                connectgaps = True,
            ), row = 1, col = 1)
        # Points at t0 and tn (start and end)
        fig.add_trace(go.Scatter(
            x=[X[0], X[-1]],
            y=[Y[0][i][0], Y[0][i][-1]],
            mode='markers', name = metaboliteLegend[i],
            marker = dict(color = metaboliteColors[i]),
            showlegend = False
        ), row = 1, col = 1)
    # Enzyme Turnover
    for i in range(len(enzymeLegend)):
        fig.add_trace(
            # The line
            go.Scatter(
                x = X, y = Y[1][i],
                mode = 'lines', name = enzymeLegend[i],
                line = dict(color = enzymeColors[i]),
                connectgaps = True,
            ), row = 1, col = 2)
        # Points at t0 and tn (start and end)
        fig.add_trace(go.Scatter(
            x=[X[0], X[-1]],
            y=[Y[1][i][0], Y[1][i][-1]],
            mode='markers', name = enzymeLegend[i],
            marker = dict(color = enzymeColors[i]),
            showlegend = False
        ), row = 1, col = 2)
    # Enzyme Percent substrate Saturation
    for i in range(len(enzymeLegend)):
        fig.add_trace(
            # The line
            go.Scatter(
                x = X, y = Y[2][i],
                mode = 'lines', name = enzymeLegend[i],
                line = dict(color = enzymeColors[i]),
                connectgaps = True,
            ), row = 2, col = 1)
        # Points at t0 and tn (start and end)
        fig.add_trace(go.Scatter(
            x=[X[0], X[-1]],
            y=[Y[2][i][0], Y[2][i][-1]],
            mode='markers', name = enzymeLegend[i],
            marker = dict(color = enzymeColors[i]),
            showlegend = False
        ), row = 2, col = 1)
    # Enzyme Percent max activity
    for i in range(len(enzymeLegend)):

        
        fig.add_trace(
            # The line
            go.Scatter(
                x = X, y = Y[3][i],
                mode = 'lines', name = enzymeLegend[i],
                line = dict(color = enzymeColors[i]),
                connectgaps = True,
            ), row = 2, col = 2)
        # Points at t0 and tn (start and end)
        fig.add_trace(go.Scatter(
            x=[X[0], X[-1]],
            y=[Y[3][i][0], Y[3][i][-1]],
            mode='markers', name = enzymeLegend[i],
            marker = dict(color = enzymeColors[i]),
            showlegend = False
        ), row = 2, col = 2)
    
    fig.update_xaxes(title_text = "Time (s)", titlefont = dict(
                family = 'Arial',
                size = 16,
                color = 'rgb(0,0,0)'
            ),
            showline=True,
            showgrid=True,
            showticklabels=True,
            linecolor='rgb(0, 0, 0)',
            linewidth=2, 
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=16,
                color='rgb(0, 0, 0)',
            ))

    fig.update_yaxes(title_text = "Concentration (uM)", 
            titlefont = dict(
                family = 'Arial',
                size = 16,
                color = 'rgb(0,0,0)'
            ),
            showline=True,
            showgrid=True,
            showticklabels=True,
            linecolor='rgb(0, 0, 0)',
            linewidth=2, 
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=16,
                color='rgb(0, 0, 0)',
            ), row = 1, col = 1)  

    fig.update_yaxes(title_text = "Concentration Turnover per Step (uM / step)", 
            titlefont = dict(
                family = 'Arial',
                size = 16,
                color = 'rgb(0,0,0)'
            ),
            showline=True,
            showgrid=True,
            showticklabels=True,
            linecolor='rgb(0, 0, 0)',
            linewidth=2, 
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=16,
                color='rgb(0, 0, 0)',
            ), row = 1, col = 2) 

    fig.update_yaxes(title_text = "Percent Substrate Saturation", 
            titlefont = dict(
                family = 'Arial',
                size = 16,
                color = 'rgb(0,0,0)'
            ),
            showline=True,
            showgrid=True,
            showticklabels=True,
            linecolor='rgb(0, 0, 0)',
            linewidth=2, 
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=16,
                color='rgb(0, 0, 0)',
            ), row = 2, col = 1) 

    fig.update_yaxes(title_text = "Percent Max Activity", 
            titlefont = dict(
                family = 'Arial',
                size = 16,
                color = 'rgb(0,0,0)'
            ),
            showline=True,
            showgrid=True,
            showticklabels=True,
            linecolor='rgb(0, 0, 0)',
            linewidth=2, 
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=16,
                color='rgb(0, 0, 0)',
            ), row = 2, col = 2) 

    fig.update_layout(
         autosize=True,   
        legend = dict(
            orientation = "h",
        ),
        plot_bgcolor='white'
        )
    
    return fig

Plotly_Simulation_Overview(
    Time = np.linspace(0, SIM_DURATION_SECONDS, TOTAL_STEPS),
    metaboliteCountData = arr2DCounts.T,
    enzymeTurnoverData = arr2DEnzymeStepTurnover.T,
    enzymeSubstrateSaturationData = arr2DEnzymePercentSubstrateSaturation.T,
    enzymePercentMaxActivityData = arr2DEnzymePercentMaxActivty.T,
    enzymeLegend = enzymeLegend,
    metaboliteLegend = metaboliteLegend
    )




In [ ]:
def Plotly_Dynamics(Title, Time, Data, Legend):
    ''' Generate a plotly plot wiht time on x axis and colored line for each data'''
    X = Time
    Y = Data
    # Temp conversion to mM
    Y = np.multiply(Y, 1000)
    Y = np.divide(Y, NA)

    colors = generateRGBList(len(Legend))

    #assert len(X) == Y.shape[-1]

    # make figure
    fig = go.Figure()

    for i in range(len(Legend)):
        if Legend[i] == "Pseudo":
            continue
        fig.add_trace(
            # The line
            go.Scatter(
                x = X, y = Y[i],
                mode = 'lines', name = Legend[i],
                line = dict(color = colors[i]),
                connectgaps = True,
                ))
        # Points at t0 and tn (start and end)
        fig.add_trace(go.Scatter(
            x=[X[0], X[-1]],
            y=[Y[i][0], Y[i][-1]],
            mode='markers', name = Legend[i],
            marker = dict(color = colors[i]),
            showlegend = False
        ))

    # Formatting 
    fig.update_layout(
        # xaxis
        xaxis=dict(
            title = "Time (s)",
            titlefont = dict(
                family = 'Arial',
                size = 16,
                color = 'rgb(0,0,0)'
            ),
            showline=True,
            showgrid=True,
            showticklabels=True,
            linecolor='rgb(0, 0, 0)',
            linewidth=2, 
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=16,
                color='rgb(0, 0, 0)',
            ),
        ),
        #  yaxis
        yaxis=dict(
            title = "Concentration (mM)",
            titlefont = dict(
                family = 'Arial',
                size = 16,
                color = 'rgb(0,0,0)'
            ),
            zeroline = True,
            showgrid=True,
            showline=True,
            showticklabels=True,
            linecolor='rgb(0, 0, 0)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=16,
                color='rgb(0, 0, 0)',
            )
        ),
        autosize=True,   
        legend = dict(
            orientation = "h",
        ),
        plot_bgcolor='white'
        )

    annotations = []
    # Adding labels
    for y_trace, label in zip(Y, Legend):
        # labeling the left_side of the plot
        annotations.append(dict(xref='paper', axref = 'x', x=0.1, y=y_trace[0]+.1,
                                    xanchor='right', yanchor='middle',
                                    text=' {:2g}mM'.format(round(y_trace[0], 2)),
                                    font=dict(family='Arial',
                                                size=16),
                                    showarrow=False))
        # labeling the right_side of the plot
        annotations.append(dict(xref='paper', axref = "x", x=0.95, y=y_trace[-1],
                                    xanchor='left', yanchor='middle',
                                    text=label +' {:2g}mM'.format(round(y_trace[-1], 2)),
                                    font=dict(family='Arial',
                                                size=16),
                                    showarrow=False))
    # Title
    annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                                xanchor='left', yanchor='bottom',
                                text='[Metabolites] over time',
                                font=dict(family='Arial',
                                            size=30,
                                            color='rgb(37,37,37)'),
                                showarrow=False))

    fig.update_layout(annotations=annotations)
    return fig

#for f6p in np.logspace( uMtoCount(1e-2), uMtoCount(1), 1):
#    def asdf(conce, f6p = f6p):
#        return ((KCAT_HEXOKINASE * CONC_HEXOKINASE / (CONC_GLUCOSE_T0 + KM_HEXOKINASE_GLUCOSE)) - (KCAT_PFK_ACTIVE* conce * (f6p / (KM_PFK_ACTIVE + f6p)) ))


def pfkConcStability(pfkConc):
    return ((KCAT_HEXOKINASE * CONC_HEXOKINASE / (CONC_GLUCOSE_T0 + KM_HEXOKINASE_GLUCOSE)) - (KCAT_PFK_ACTIVE * pfkConc * (uMtoCount(15) / (KM_PFK_ACTIVE + uMtoCount(15))) ))

pfkConc = np.geomspace(uMtoCount(1e-17), uMtoCount(1e5), 24)    

conc = 4000


for f6pConc in np.geomspace(uMtoCount(1e-6), uMtoCount(1e6), 13): #1 nM to 1M
    print('-------------------------------------------')  
    print(f'[f6p] = {round(counttouM(f6pConc),6)} uM')
    print('-------------------------------------------')  
    def pfkConcStability(pfkConc = 4000):
        return ((KCAT_HEXOKINASE * CONC_HEXOKINASE / (CONC_GLUCOSE_T0 + KM_HEXOKINASE_GLUCOSE)) - (KCAT_PFK_ACTIVE * pfkConc * (f6pConc / (KM_PFK_ACTIVE + f6pConc)) ))
    #for conc in pfkConc:
    print(fixed_point(pfkConcStability, conc))